##### 5) Using sklearn, considering a 10-fold stratified cross validation (random=0), plot the cumulative testing confusion matrices of kNN (uniform weights, k = 5, Euclidean distance) and Naive Bayes (Gaussian assumption). Use all remaining classifier parameters as default.

Read data:

In [13]:
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff

data = loadarff('pd_speech.arff')
df = pd.DataFrame(data[0])
df = df.dropna()
df['class'] = df['class'].str.decode('utf-8')

X = df.drop('class', axis=1)
Y = df['class']

Creates the 10-fold cross validator, the KNN and the Gaussian Naive Bayes:

In [21]:
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0) 

knn = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='euclidean')
nb = GaussianNB()

for train, test in skf.split(X, Y):
    knn.fit(X.iloc[train], Y.iloc[train])
    nb.fit(X.iloc[train], Y.iloc[train])
    
    
    
    


TypeError: int() argument must be a string, a bytes-like object or a number, not 'DataFrame'


---  

##### 6) Using scipy, test the hypothesis “𝑘NN is statistically superior to Naïve Bayes regarding accuracy”, asserting whether is true.

In [ ]:
#TODO


---

##### 7) Enumerate three possible reasons that could underlie the observed differences in predictive accuracy between 𝑘NN and Naïve Bayes.

Reasons:  
1. #TODO
2. #TODO  
3. #TODO  

---

TODO Cenas

- [ ]: Diferença entre [KNeighborsRegressor e KNeighborsClassifier](https://stackoverflow.com/questions/52794075/sklearn-kneighborsregressor-vs-kneighborsclassifer)